In [ ]:
#Autoregression Clustering- Let's use nearby values to determine where we are going. 
#Dumb probabilities- this will get us an answer

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

In [3]:
df=pd.read_csv(r"C:\Users\Daniel\3D Objects\Nasa_maitenance\Challenge_Data\train.txt",delim_whitespace=True, header=None)
traindf=pd.read_csv(r"C:\Users\Daniel\3D Objects\Nasa_maitenance\Challenge_Data\train.txt",delim_whitespace=True, header=None)
validdf=pd.read_csv(r"C:\Users\Daniel\3D Objects\Nasa_maitenance\Challenge_Data\test.txt",delim_whitespace=True, header=None)
finaldf=pd.read_csv(r"C:\Users\Daniel\3D Objects\Nasa_maitenance\Challenge_Data\final_test.txt",delim_whitespace=True, header=None)
print("Dataframe sizes: {},{},{}".format(traindf.shape,validdf.shape,finaldf.shape))

Dataframe sizes: (45918, 26),(29820, 26),(55156, 26)


In [4]:
#Can scale all data at once. Normally we split the data up going forward, but this was already done. 
#We could build a function that outputs a matrix
scaler = StandardScaler()
alldf=pd.concat([traindf,validdf,finaldf])
print(alldf[2:].head())
alldf[2:]=scaler.fit_transform(alldf[2:])


   0   1        2       3     4       5       6        7        8     9   ...  \
2   1   3  34.9986  0.8401  60.0  449.44  555.42  1368.17  1122.49  5.48  ...   
3   1   4  20.0031  0.7005   0.0  491.19  607.03  1488.44  1249.18  9.35  ...   
4   1   5  42.0041  0.8405  40.0  445.00  549.52  1354.48  1124.32  3.91  ...   
5   1   6  20.0032  0.7017   0.0  491.19  607.37  1480.46  1258.90  9.35  ...   
6   1   7  41.9998  0.8400  40.0  445.00  549.57  1354.43  1131.44  3.91  ...   

       16       17       18      19    20   21    22     23     24       25  
2  183.26  2387.95  8063.84  9.3557  0.02  334  2223  100.0  14.83   8.8555  
3  314.84  2388.07  8052.30  9.2231  0.02  364  2324  100.0  24.42  14.7832  
4  130.44  2387.89  8083.67  9.2986  0.02  330  2212  100.0  10.99   6.4025  
5  315.36  2388.05  8053.17  9.2276  0.02  364  2324  100.0  24.44  14.7019  
6  130.16  2387.88  8082.01  9.3753  0.02  331  2212  100.0  10.53   6.4254  

[5 rows x 26 columns]


In [5]:
#borrow function from before
def listvals(value):
    value=int(value-1)
    return life_list[value]

lifedf=alldf.groupby(0).agg({1:['max']})
life_list=lifedf[1,"max"].tolist()


alldf["Degrade"]=(alldf[0].apply(listvals)-alldf[1])/alldf[0].apply(listvals)
alldf["TotalLife"]=alldf[0].apply(listvals)


In [7]:
#Split back up
place1=45918+29820
place2=place1+55156
traindf_sc=alldf.iloc[0:45918,:]
validdf_sc=alldf.iloc[place1:place2,:]
finaldf_sc=alldf.iloc[place2:,:]

In [20]:
#Dumb distance definition
class recursive_distance():

    def __init__(self, totalvec,decay):
        self.lifetotals=totalvec
        self.iterations=0.0
        self.compare=len(totalvec)
        self.decay=decay
        
    def distance(self,df,vec):
        self.total=0
        current_rows=self.compare*self.iterations
        current_compare=df.iloc[z+current_rows]
        for z in range(0,self.compare):
            update=cdist(df.iloc[z+current_rows].to_numpy(), vec, metric='euclidean')
            self.distancevec[z]=self.decay*self.distancevec[z]+update
            self.total+=self.distancevec[z]
            
    #select return function to determine estimate of remaining life
    def max_based(self):
        return self.lifetotals[self.distancevec[z].index(max(self.distancevec))]
    def total_based(self):
        final_value=0.0
        count=0
        for x in self.distancevec:
            weight=x/self.total
            final_value+=weight*self.lifetotals[count]
            count+=1

In [18]:
grouped_df = validdf_sc.groupby(0)    
validationset=[grouped_df.get_group(x) for x in grouped_df.groups]
validationset[0].head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,Degrade,TotalLife
0,-1.429151,-1.424903,-0.948590,-1.035900,-0.894683,0.613253,0.684198,0.755057,0.850539,0.688592,...,0.765480,-0.910094,1.456036,0.829652,0.622739,0.419797,0.780913,0.773012,3.563170,0.555914
1,-1.429151,-1.408929,-1.625479,-1.840378,1.566864,1.733955,1.693069,1.642907,1.693313,1.820517,...,0.759437,-1.192080,1.456036,1.585537,1.096522,0.419797,1.834363,1.832782,3.534435,0.555914
2,-1.429151,-1.392955,0.742679,0.864190,0.336090,-0.885431,-0.642306,-0.574268,-0.637678,-0.702847,...,-0.046236,-0.022157,-0.686796,-0.538140,-0.036436,0.419797,-0.593723,-0.615212,3.505700,0.555914
3,-1.429151,-1.376980,-0.272025,0.414609,-1.510070,0.694222,0.753830,0.577260,0.450756,0.365579,...,-0.047065,-0.113309,-0.686796,0.613685,0.657071,0.419797,0.359735,0.376112,3.476965,0.555914
4,-1.429151,-1.361006,1.216722,0.865478,-0.279296,-1.053423,-0.797641,-0.607640,-0.665785,-1.136292,...,0.197717,0.095782,-0.686796,-0.610129,-0.111967,0.419797,-1.021971,-1.008241,3.448229,0.555914


In [11]:
traindf_sc.iloc[1].to_numpy()

array([1.00000000e+00, 2.00000000e+00, 1.50000000e-03, 3.00000000e-04,
       1.00000000e+02, 5.18670000e+02, 6.42130000e+02, 1.58455000e+03,
       1.40396000e+03, 1.46200000e+01, 2.16100000e+01, 5.53670000e+02,
       2.38801000e+03, 9.04576000e+03, 1.30000000e+00, 4.72900000e+01,
       5.21810000e+02, 2.38815000e+03, 8.13287000e+03, 8.39070000e+00,
       3.00000000e-02, 3.91000000e+02, 2.38800000e+03, 1.00000000e+02,
       3.89900000e+01, 2.33619000e+01, 5.72260758e-02, 2.12139936e+00])

In [23]:
#feed grouped by second column
distance_system=recursive_distance(life_list,0.95)
sorted_train=traindf_sc.groupby(1)
for y in range(0,50):
    distance_system.distance(sorted_train[:,2:26],z.iloc[y,2:26])
    distance_system.max_based()


AttributeError: 'DataFrameGroupBy' object has no attribute 'iloc'